In [2]:
#!/usr/bin/env python2
# -*- coding: utf-8 -*-
"""
Created on Sun Oct 30 20:11:19 2016

@author: stephen
"""
 
from __future__ import print_function
 
from keras.models import Model
from keras.utils import np_utils
import numpy as np
import pandas as pd
import keras 
from keras.callbacks import ReduceLROnPlateau
      
def readucr(filename):
    data = np.loadtxt(filename, delimiter = ',')
    Y = data[:,0]
    X = data[:,1:]
    return X, Y
  
nb_epochs = 2000
foder = u"E:/Jupyter File/UCR_TS_Archive_2015/"

#flist = ['Adiac', 'Beef', 'CBF', 'ChlorineConcentration', 'CinC_ECG_torso', 'Coffee', 'Cricket_X', 'Cricket_Y', 'Cricket_Z', 
#'DiatomSizeReduction', 'ECGFiveDays', 'FaceAll', 'FaceFour', 'FacesUCR', '50words', 'FISH', 'Gun_Point', 'Haptics', 
#'InlineSkate', 'ItalyPowerDemand', 'Lighting2', 'Lighting7', 'MALLAT', 'MedicalImages', 'MoteStrain', 'NonInvasiveFatalECG_Thorax1', 
#'NonInvasiveFatalECG_Thorax2', 'OliveOil', 'OSULeaf', 'SonyAIBORobotSurface', 'SonyAIBORobotSurfaceII', 'StarLightCurves', 'SwedishLeaf', 'Symbols', 
#'synthetic_control', 'Trace', 'TwoLeadECG', 'Two_Patterns', 'uWaveGestureLibrary_X', 'uWaveGestureLibrary_Y', 'uWaveGestureLibrary_Z', 'wafer', 'WordsSynonyms', 'yoga']

flist  = ['Adiac']
for each in flist:
    fname = each
    x_train, y_train = readucr(foder+fname+'/'+fname+'_TRAIN')
    x_test, y_test = readucr(foder+fname+'/'+fname+'_TEST')
    nb_classes = len(np.unique(y_test))
    batch_size = min(x_train.shape[0]/10, 16)
    
    y_train = (y_train - y_train.min())/(y_train.max()-y_train.min())*(nb_classes-1)
    y_test = (y_test - y_test.min())/(y_test.max()-y_test.min())*(nb_classes-1)
    
    
    Y_train = np_utils.to_categorical(y_train, nb_classes)
    Y_test = np_utils.to_categorical(y_test, nb_classes)
    
    x_train_mean = x_train.mean()
    x_train_std = x_train.std()
    x_train = (x_train - x_train_mean)/(x_train_std)
     
    x_test = (x_test - x_train_mean)/(x_train_std)
    x_train = x_train.reshape(x_train.shape + (1,1,))
    x_test = x_test.reshape(x_test.shape + (1,1,))

    x = keras.layers.Input(x_train.shape[1:])
#    drop_out = Dropout(0.2)(x)
    conv1 = keras.layers.Conv2D(256, 5, 1, border_mode='same')(x)
    conv1 = keras.layers.normalization.BatchNormalization()(conv1)
    conv1 = keras.layers.Activation('relu')(conv1)
    
#    drop_out = Dropout(0.2)(conv1)
    conv2 = keras.layers.Conv2D(128, 3, 1, border_mode='same')(conv1)
    conv2 = keras.layers.normalization.BatchNormalization()(conv2)
    conv2 = keras.layers.Activation('relu')(conv2)
     
    full = keras.layers.pooling.GlobalAveragePooling2D()(conv2)    
    out = keras.layers.Dense(nb_classes, activation='softmax')(full)
    
    
    model = Model(input=x, output=out)
     
    optimizer = keras.optimizers.Adam()
    model.compile(loss='categorical_crossentropy',
                  optimizer=optimizer,
                  metrics=['accuracy'])
     
    reduce_lr = ReduceLROnPlateau(monitor = 'loss', factor=0.5,
                      patience=50, min_lr=0.0001) 
    hist = model.fit(x_train, Y_train, batch_size=batch_size, nb_epoch=nb_epochs,
              verbose=1, validation_data=(x_test, Y_test), callbacks = [reduce_lr])
    #Print the testing results which has the lowest training loss.
    log = pd.DataFrame(hist.history)
    print(log.loc[log['loss'].idxmin]['loss'], log.loc[log['loss'].idxmin]['val_acc'])

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:58: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(256, (5, 1), padding="same")`
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:63: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 1), padding="same")`
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:71: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:81: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 390 samples, validate on 391 samples
Epoch 1/2000
390/390 [==============================] - 5s - loss: 3.6667 - acc: 0.0103 - val_loss: 3.6182 - val_acc: 0.0435
Epoch 2/2000
390/390 [==============================] - 3s - loss: 3.5816 - acc: 0.0359 - val_loss: 3.6402 - val_acc: 0.0281
Epoch 3/2000
390/390 [==============================] - 3s - loss: 3.5619 - acc: 0.0513 - val_loss: 3.7101 - val_acc: 0.0281
Epoch 4/2000
390/390 [==============================] - 3s - loss: 3.5517 - acc: 0.0641 - val_loss: 3.8625 - val_acc: 0.0281
Epoch 5/2000
390/390 [==============================] - 3s - loss: 3.5422 - acc: 0.0897 - val_loss: 4.1253 - val_acc: 0.0281
Epoch 6/2000
390/390 [==============================] - 3s - loss: 3.5242 - acc: 0.1103 - val_loss: 4.4318 - val_acc: 0.0281
Epoch 7/2000
390/390 [==============================] - 3s - loss: 3.4999 - acc: 0.0795 - val_loss: 5.1615 - val_acc: 0.0281
Epoch 8/2000
390/390 [==============================] - 3s - loss: 3.4719 - acc

390/390 [==============================] - 3s - loss: 2.1960 - acc: 0.3821 - val_loss: 4.4134 - val_acc: 0.0614
Epoch 66/2000
390/390 [==============================] - 3s - loss: 2.1832 - acc: 0.4256 - val_loss: 3.5922 - val_acc: 0.0946
Epoch 67/2000
390/390 [==============================] - 3s - loss: 2.2232 - acc: 0.4308 - val_loss: 2.5799 - val_acc: 0.2634
Epoch 68/2000
390/390 [==============================] - 3s - loss: 2.1020 - acc: 0.4359 - val_loss: 2.7580 - val_acc: 0.1637
Epoch 69/2000
390/390 [==============================] - 3s - loss: 2.1694 - acc: 0.4231 - val_loss: 2.4734 - val_acc: 0.2302
Epoch 70/2000
390/390 [==============================] - 3s - loss: 2.1412 - acc: 0.4128 - val_loss: 2.8493 - val_acc: 0.1023
Epoch 71/2000
390/390 [==============================] - 3s - loss: 2.0750 - acc: 0.4564 - val_loss: 2.9154 - val_acc: 0.0793
Epoch 72/2000
390/390 [==============================] - 3s - loss: 2.0758 - acc: 0.4692 - val_loss: 2.4519 - val_acc: 0.2404
Epoch 

390/390 [==============================] - 5s - loss: 1.6190 - acc: 0.5641 - val_loss: 3.4523 - val_acc: 0.1535
Epoch 130/2000
390/390 [==============================] - 5s - loss: 1.6079 - acc: 0.5436 - val_loss: 6.0122 - val_acc: 0.1049
Epoch 131/2000
390/390 [==============================] - 5s - loss: 1.5679 - acc: 0.5641 - val_loss: 2.9696 - val_acc: 0.1944
Epoch 132/2000
390/390 [==============================] - 5s - loss: 1.5054 - acc: 0.5923 - val_loss: 4.2413 - val_acc: 0.1432
Epoch 133/2000
390/390 [==============================] - 5s - loss: 1.5605 - acc: 0.5667 - val_loss: 3.8269 - val_acc: 0.1355
Epoch 134/2000
390/390 [==============================] - 5s - loss: 1.6096 - acc: 0.5538 - val_loss: 2.8975 - val_acc: 0.1918
Epoch 135/2000
390/390 [==============================] - 5s - loss: 1.6122 - acc: 0.5744 - val_loss: 3.1666 - val_acc: 0.1253
Epoch 136/2000
390/390 [==============================] - 5s - loss: 1.5562 - acc: 0.5949 - val_loss: 4.8816 - val_acc: 0.1176

390/390 [==============================] - 5s - loss: 1.2980 - acc: 0.6077 - val_loss: 4.4829 - val_acc: 0.0921
Epoch 194/2000
390/390 [==============================] - 5s - loss: 1.2840 - acc: 0.6333 - val_loss: 2.7751 - val_acc: 0.2506
Epoch 195/2000
390/390 [==============================] - 5s - loss: 1.2699 - acc: 0.6154 - val_loss: 2.6473 - val_acc: 0.2276
Epoch 196/2000
390/390 [==============================] - 5s - loss: 1.2376 - acc: 0.6410 - val_loss: 3.0190 - val_acc: 0.1739
Epoch 197/2000
390/390 [==============================] - 5s - loss: 1.2863 - acc: 0.6410 - val_loss: 2.4245 - val_acc: 0.2481
Epoch 198/2000
390/390 [==============================] - 5s - loss: 1.3986 - acc: 0.6231 - val_loss: 4.8473 - val_acc: 0.1688
Epoch 199/2000
390/390 [==============================] - 5s - loss: 1.3172 - acc: 0.5974 - val_loss: 7.3669 - val_acc: 0.1714
Epoch 200/2000
390/390 [==============================] - 5s - loss: 1.2841 - acc: 0.6513 - val_loss: 5.5163 - val_acc: 0.2020

...

390/390 [==============================] - 4s - loss: 0.2350 - acc: 0.9487 - val_loss: 0.5836 - val_acc: 0.8210
Epoch 1730/2000
390/390 [==============================] - 4s - loss: 0.2406 - acc: 0.9333 - val_loss: 0.6030 - val_acc: 0.8286
Epoch 1731/2000
390/390 [==============================] - 4s - loss: 0.2260 - acc: 0.9385 - val_loss: 0.6069 - val_acc: 0.8159
Epoch 1732/2000
390/390 [==============================] - 4s - loss: 0.2484 - acc: 0.9205 - val_loss: 0.5922 - val_acc: 0.8235
Epoch 1733/2000
390/390 [==============================] - 4s - loss: 0.2336 - acc: 0.9385 - val_loss: 0.6804 - val_acc: 0.8133
Epoch 1734/2000
390/390 [==============================] - 4s - loss: 0.2536 - acc: 0.9333 - val_loss: 0.7662 - val_acc: 0.7647
Epoch 1735/2000
390/390 [==============================] - 4s - loss: 0.2455 - acc: 0.9282 - val_loss: 0.6557 - val_acc: 0.8056
Epoch 1736/2000
390/390 [==============================] - 4s - loss: 0.2312 - acc: 0.9487 - val_loss: 0.6018 - val_acc:

390/390 [==============================] - 4s - loss: 0.2402 - acc: 0.9282 - val_loss: 0.5812 - val_acc: 0.8338
Epoch 1794/2000
390/390 [==============================] - 4s - loss: 0.2507 - acc: 0.9333 - val_loss: 0.6042 - val_acc: 0.8261
Epoch 1795/2000
390/390 [==============================] - 4s - loss: 0.2497 - acc: 0.9231 - val_loss: 0.6165 - val_acc: 0.8107
Epoch 1796/2000
390/390 [==============================] - 4s - loss: 0.2482 - acc: 0.9308 - val_loss: 0.5975 - val_acc: 0.8210
Epoch 1797/2000
390/390 [==============================] - 4s - loss: 0.2274 - acc: 0.9436 - val_loss: 0.6136 - val_acc: 0.8210
Epoch 1798/2000
390/390 [==============================] - 4s - loss: 0.2273 - acc: 0.9359 - val_loss: 0.5985 - val_acc: 0.8184
Epoch 1799/2000
390/390 [==============================] - 4s - loss: 0.2275 - acc: 0.9282 - val_loss: 0.6062 - val_acc: 0.8312
Epoch 1800/2000
390/390 [==============================] - 4s - loss: 0.2507 - acc: 0.9256 - val_loss: 0.6616 - val_acc:

390/390 [==============================] - 4s - loss: 0.2491 - acc: 0.9308 - val_loss: 0.8485 - val_acc: 0.7468
Epoch 1858/2000
390/390 [==============================] - 4s - loss: 0.2836 - acc: 0.9205 - val_loss: 0.6854 - val_acc: 0.7903
Epoch 1859/2000
390/390 [==============================] - 4s - loss: 0.2394 - acc: 0.9333 - val_loss: 0.6687 - val_acc: 0.7954
Epoch 1860/2000
390/390 [==============================] - 4s - loss: 0.2611 - acc: 0.9154 - val_loss: 0.6403 - val_acc: 0.8159
Epoch 1861/2000
390/390 [==============================] - 4s - loss: 0.2207 - acc: 0.9256 - val_loss: 0.6216 - val_acc: 0.8389
Epoch 1862/2000
390/390 [==============================] - 4s - loss: 0.2610 - acc: 0.9154 - val_loss: 0.6839 - val_acc: 0.8133
Epoch 1863/2000
390/390 [==============================] - 4s - loss: 0.2429 - acc: 0.9513 - val_loss: 0.7538 - val_acc: 0.7724
Epoch 1864/2000
390/390 [==============================] - 4s - loss: 0.2587 - acc: 0.9308 - val_loss: 0.6687 - val_acc:

390/390 [==============================] - 4s - loss: 0.2252 - acc: 0.9359 - val_loss: 0.6565 - val_acc: 0.7903
Epoch 1922/2000
390/390 [==============================] - 4s - loss: 0.2305 - acc: 0.9436 - val_loss: 0.6298 - val_acc: 0.8133
Epoch 1923/2000
390/390 [==============================] - 4s - loss: 0.2442 - acc: 0.9359 - val_loss: 0.5927 - val_acc: 0.8235
Epoch 1924/2000
390/390 [==============================] - 4s - loss: 0.2395 - acc: 0.9256 - val_loss: 0.5973 - val_acc: 0.8261
Epoch 1925/2000
390/390 [==============================] - 4s - loss: 0.2515 - acc: 0.9333 - val_loss: 0.6416 - val_acc: 0.8312
Epoch 1926/2000
390/390 [==============================] - 4s - loss: 0.2354 - acc: 0.9231 - val_loss: 0.5896 - val_acc: 0.8184
Epoch 1927/2000
390/390 [==============================] - 4s - loss: 0.2365 - acc: 0.9359 - val_loss: 0.7181 - val_acc: 0.7903
Epoch 1928/2000
390/390 [==============================] - 4s - loss: 0.2312 - acc: 0.9333 - val_loss: 0.6137 - val_acc:

390/390 [==============================] - 4s - loss: 0.2648 - acc: 0.9179 - val_loss: 0.6071 - val_acc: 0.8210
Epoch 1986/2000
390/390 [==============================] - 4s - loss: 0.2223 - acc: 0.9333 - val_loss: 0.6794 - val_acc: 0.8056
Epoch 1987/2000
390/390 [==============================] - 4s - loss: 0.2256 - acc: 0.9231 - val_loss: 0.6112 - val_acc: 0.8261
Epoch 1988/2000
390/390 [==============================] - 4s - loss: 0.2241 - acc: 0.9385 - val_loss: 0.6302 - val_acc: 0.8056
Epoch 1989/2000
390/390 [==============================] - 4s - loss: 0.2642 - acc: 0.9179 - val_loss: 0.6023 - val_acc: 0.8184
Epoch 1990/2000
390/390 [==============================] - 4s - loss: 0.2302 - acc: 0.9436 - val_loss: 0.6361 - val_acc: 0.8005
Epoch 1991/2000
390/390 [==============================] - 4s - loss: 0.2109 - acc: 0.9385 - val_loss: 0.6499 - val_acc: 0.7877
Epoch 1992/2000
390/390 [==============================] - 4s - loss: 0.2205 - acc: 0.9487 - val_loss: 0.5982 - val_acc: